# YOLO model:
The YOLO model has been trained using a C implementation called darknet. this example uses a fork from AlexeyAB, that allows training YOLO using darknet. Therefore, the following are the scripts to call to perform the different operations.

First, make sure submodules are initiated:

In [1]:
!git submodule init

Compile darknet for use on the HPC:

In [ ]:
!./compile_darknet.sh

## Training

First, from within the scripts/YOLO folder, call fish_label.py
This generates the training and validation text files.

After having generated the json files from the front-end, call add_from_network.py to include the new images to the training set. The validation set stays the same.

To train using the HPC, queue up for use with 2 graphics cards using:

In [ ]:
!bsub < queueYolo

In [ ]:
!./queueYolo

## Inference

To perform Inference on all the extracted images from all the videos, set the used weight file in queYolo_inference and the list of files to run inference on in the same file and then queue up or run interactively using the following:

In [ ]:
!bsub < queueYolo_inference

In [ ]:
!./queueYolo_inference

## mAP on set
To calculate the mean average precision on the validation set, change the weight filed pointed to in queueYolo_map to point to the desired .weight file and run:

In [ ]:
!bsub < queueYolo_map

In [ ]:
!./queueYolo_map

## Generate images with bounding boxes from the result file


The following takes a json file and reads in all the detections. It then takes the cases where a minimum of one object was detected by one of two networks. It draws a bounding box and saves the image in the folder that the code was run from. The first network is drawn in red while the second network is drawn in blue.

To do this, use the inference script to output the two json files.

In [ ]:
from PIL import Image, ImageDraw

import json

with open("/work1/s154227/videos/result_first_500.json") as file:
	dict_first = json.load(file)
with open("/work1/s154227/videos/result_second_500.json") as file:
	dict_second = json.load(file)

for i in range(500):
	if len(dict_first[i]["objects"]) > 0 or len(dict_second[i]["objects"]) > 0:
		path = dict_first[i]["filename"]
		im = Image.open(path)
		draw = ImageDraw.Draw(im)
		for obj in dict_first[i]["objects"]:
			coor = obj["relative_coordinates"]
			x0 = 1280 * coor["center_x"] - 1280*(float(coor["width"])/2)
			x1 = 1280 * coor["center_x"] + 1280*(float(coor["width"])/2)
			y0 = 720 * coor["center_y"] - 720*float(coor["height"])/2
			y1 = 720 * coor["center_y"] + 720*float(coor["height"])/2
			draw.rectangle((x0,y0,x1,y1), outline = (255,0,0))
			draw.text((x0, y1),str(obj["confidence"]), fill = (255,0,0) )
		for obj in dict_second[i]["objects"]:
			coor = obj["relative_coordinates"]
			x0 = 1280 * coor["center_x"] - 1280*float(coor["width"])/2
			x1 = 1280 * coor["center_x"] + 1280*float(coor["width"])/2
			y0 = 720 * coor["center_y"] - 720*float(coor["height"])/2
			y1 = 720 * coor["center_y"] + 720*float(coor["height"])/2
			draw.rectangle((x0+2,y0+2,x1+2,y1+2), outline = (0,0,255))
			draw.text((x0, y1),str(obj["confidence"]), fill = (0,0,255) )
		im.save(path.split("/")[-1])